# **<span style="font-family: 'Palatino Linotype', serif;">🫂 O bom vizinho sempre tem algo a emprestar...</span>**

*<span style="font-family: 'Angilla Tattoo'">Habitamos em um humilde vilarejo que, se batida a porta, não há conhecimento acerca do nível de bondade do ser que nela bate. Dito isto, você confiaria mesmo nas palavras de um vizinho? </span>*
<div align="center">
    <img src="assets\mapas\2.png" alt="Mapa: A Batalha Final">
    <figcaption>Descobrindo sobre os vizinhos</figcaption>
</div>

---

**Trabalho Final: 🏘️Previsão pelo modelo dos k-NN vizinhos**
==========================================================

**Autores:** Sepulcro de Delfos
* Ana Luz
* Caio Ruas
* Caio Matheus
* Giovana Martins

## 🚩 **Introdução**

Este é o segundo de uma série de 4 notebooks que compõem o trabalho final da disciplina de Aprendizado de Máquina. Recomenda-se a leitura do primeiro notebook, que contém a descrição do problema e a análise exploratória dos dados, ele pode ser acessado aqui [link](https://github.com/CaioRuas24010/SepulcroDeDelfos/blob/main/A%20Batalha%20Contra%20Dragao/introducao.ipynb).

O presente trabalho tem como objetivo a implementação de um modelo de classificação baseado no algoritmo dos k-vizinhos mais próximos (k-NN) para prever o *band gap* de materiais para aplicação em células solares.

**Observação** No caderno anterior foi realizada uma análise com o `optuna` em relação à atributos anteriomente denominados como *derivados* (Numero atômico médio, dentre outros) e *diretos* (Fração molar por elemento), e foi constatado que os atributos *diretos* geraram erros menores para o modelo da floresta aleatória. Apesar disso, como citado no caderno anterior, os atributos *diretos* são mais difíceis de serem utilizados e, por isso, optamos por utilizar os atributos *derivados* para o modelo do k-NN.

## 👷‍♂️ **Alicerces do conhecimento**

O primeiro passo a ser feito é a importação das bibliotecas necessárias para que o modelo se torne prático e funcional:

In [1]:
import pandas as pd
import plotly.express as px

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error

## 🎲 **Obtenção e manipulação dos dados**

Nenhuma casa é construída sem seus tijolos. Dito isto, vamos coletar os dados necessários para que o modelo possa ser desenvolvido e a previsão ocorra. Os dados já foram devidamente apresentados no notebook introdutório, por isso pularemos etapas descritivas ou exploratórias.

In [2]:
df = pd.read_csv("materials_trabalhado.csv")
print(df)

      Unnamed: 0.1  Unnamed: 0 material_id  theoretical  band_gap   density  \
0                0           0    mp-28967        False    0.7792  5.022717   
1                1           1   mp-766094         True    2.8980  3.764366   
2                2           2    mp-36577         True    1.7212  3.094976   
3                3           3  mp-1102092        False    2.0944  2.901260   
4                4           4   mp-720391        False    7.4812  1.992908   
...            ...         ...         ...          ...       ...       ...   
6242          6242        6242    mp-18741        False    2.7642  3.069872   
6243          6243        6243    mp-20078        False    1.0749  7.854668   
6244          6244        6244     mp-5504        False    4.3974  4.244726   
6245          6245        6245   mp-776470         True    1.0064  7.854668   
6246          6246        6246   mp-752397         True    3.7685  3.739078   

          volume      symmetry  dieletric_constant 

Para a implementação do regressor k-NN, precisamos de dados que sejam bem ajustados, tanto no âmbito de dimensões, quanto na escolha de atributos e target que serão utilizados posteriormente para o processo de previsão.
Para isso, foram escolhidos - para previsão do `band gap` - os atributos: 
* `Densidade`
* `Energia de formação por átomo`
* `Número de sítios`
* `Simetria Codificada`
* `Constante dielétrica`
* `Número Atômico Médio`
* `Densidade Atômica`

Sendo essas algumas propriedades que influenciam no valor do band gap e ajudam na nossa previsão:

In [3]:
atributos = df[["density", "formation_energy_per_atom", "nsites", "symmetry_encoded", "dieletric_constant", "average_atomic_number", "densidade_atômica"]]
target = df["band_gap"]

X = atributos
y = target

## ⚖️ **Jogue os dados e a sua vida será normal**

Para que a previsão a partir dos atibutos *não seja enviesada* por uma possível diferença de escala dos valores - visto que o algoritmo dos k-NN vizinhos se baseia em medidas de distância -, é necessário que os dados sejam **normalizados**, distribuindo de maneira justa os pesos entre os atributos escolhidos e resultando numa previsão mais factível.

In [4]:
normalizador = StandardScaler()
normalizador.fit(X)
X_normalizado = normalizador.transform(X)

## 👤 **E bate à porta o vizinho...**

Se há alguma batida na porta, nada mais justo que atender, mas quem disse que seria só um vizinho?

Escolhe-se, portanto, um valor referente ao número de vizinhos analisados e importa-se a função *KNeighborsRegressor* da biblioteca `Numpy` para fazer a previsão.

Além disso, para o processo de ajuste dos dados (cálculo das distâncias, por exemplo), é preciso fazer um *fit* nos dados já normalizados de atributos e target.


In [5]:
NUM_VIZINHOS = 1000
knn = KNeighborsRegressor(n_neighbors=NUM_VIZINHOS)
knn.fit(X_normalizado, y)

KNeighborsRegressor(n_neighbors=1000)

Posterior ao ajuste de dados, precisa-se de um **palpite inicial** que será utilizado para a realização da **previsão** do valor do **band gap**, de forma que cada item ordenado da lista se refira às colunas escolhidas como atributos durante o processo de manipulação de dados, precisando ser normalizado pelos mesmos motivos de escala.

In [6]:
palpite_x = [
    [5,-1,300,0.8,5,33,0.05],
]
palpite_x_normalizado = normalizador.transform(palpite_x)

C:\Users\caio24010\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Após a definição do palpite, é hora de realizar a **previsão**, usando a função `predict` com os dados normalizados obtidos anteriormente.

In [7]:
previsao = knn.predict(palpite_x_normalizado)
print(previsao)

[2.8763292]


## 📏 **Métrica do Erro Quadrado Médio (MSE)**

Para a análise da **eficiência** do modelo dos k-NN vizinhos, pode-se usar a métrica de performance do MSE, que pode ser dada por: 

$$
\mathrm{MSE} = \frac{1}{n}\sum_{i=1}^{n} (y_i - \hat{y}_i)^2
$$

In [8]:
previsao_y = knn.predict(X)
MSE = mean_squared_error(y, previsao_y)
print(f"O MSE do modelo analisando {NUM_VIZINHOS} vizinhos foi de {MSE}")

C:\Users\caio24010\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but KNeighborsRegressor was fitted without feature names
  warnings.warn(


O MSE do modelo analisando 1000 vizinhos foi de 3.3371470441863798


## 🫚 **Métrica da Raiz do Erro Quadrático Médio (RMSE)**
Outra métrica que pode ser utilizada como métrica de performance é o RMSE, que é representada pela raiz quadrada do valor do MSE, como pode ser visto na equação abaixo:

$$
\mathrm{RMSE} = \sqrt{\frac{1}{n}\sum_{i=1}^{n} (y_i - \hat{y}_i)^2}
$$

In [9]:
RMSE = root_mean_squared_error(y, previsao_y)
print(f"O RMSE do modelo analisando {NUM_VIZINHOS} vizinhos foi de {RMSE}")

O RMSE do modelo analisando 1000 vizinhos foi de 1.8267859875164303


## 🧮 **Comparação entre média e valor previsto**
Para análise dos dados, também é válido comparar os dados já conhecidos com os dados agora previstos, para isso, calculamos a diferença entre as médias:

In [10]:
media_band_gap = df["band_gap"].mean()
media_band_gap_previsao = previsao_y.mean()
print(f"A média dos valores de band gap encontrados no DataFrame é de {media_band_gap}")
print(f"A média dos valores previstos do band gap pelo modelo foi de {media_band_gap_previsao}")
print()
print(f"Portanto, a diferença entre as médias encontradas foi de {media_band_gap - media_band_gap_previsao}")

A média dos valores de band gap encontrados no DataFrame é de 2.290884184408516
A média dos valores previstos do band gap pelo modelo foi de 1.3674861237233873

Portanto, a diferença entre as médias encontradas foi de 0.9233980606851289


## 🔎 **A vida é feita de escolhas (e alguns erros)...**

*"Dizem que todos nós somos feitos de erros, mas será que os erros são feitos de todos nós?"*
-Autor desconhecido

Com isso em mente, calcularemos e plotaremos um gráfico a partir dos erros encontrados no modelo.

In [11]:
erros = y - previsao_y

fig = px.histogram(x=erros, nbins=40, title="Histograma dos erros do KNN")
fig.show()

A partir da análise do histograma foi possível observar uma elevada distribuição dos erros, que sugere a efetividade do modelo k-NN, em geral, de realizar boas estimativas a partir dos atributos escolhidos e normalizados, de modo a diminuir o que poderia ser um erro muito grande.

## ⚰️ **Conclusão**

Neste trabalho, foi desenvolvido um algoritmo baseado no método dos k-vizinhos mais próximos  para prever o valor do band gap de materiais, utilizando atributos de dados físico-químicos normalizados para busca de um valor de band gap a partir de um palpite inicial. 

A avaliação do modelo foi realizada a partir das métricas de performance do Erro Quadrático Médio (MSE) e  da Raiz do Erro Quadrático Médio  (RMSE). O MSE quantifica a média dos erros ao quadrado entre os valores previstos e os valores reais do band gap, enquanto o RMSE fornece uma interpretação mais intuitiva ao trazer o erro para a mesma unidade do band gap.

Esses resultados sugerem que, ao combinar a simetria cristalina, propriedades atômicas e físicas, como a densidade e a energia de formação, é possível realizar uma previsão do valor do band gap de um material com razoável precisão, dependendo de alguns ajustes durante a manipulação dos dados, como a escolha do valor dos vizinhos, a escolha de atributos ou o uso de técnicas de otimização, que influenciam no desempenho medido.

## 📖 **Referências**

CASSAR, Daniel. **Aprendizado de máquina, k-NN e métricas**. 2024. Material de Aula.

SCIKIT-LEARN DEVELOPERS. **sklearn.neighbors.KNeighborsRegressor**. Disponível em: [https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html).

STAT QUEST WITH JOSH STARMER. **StatQuest: K-nearest neighbors, Clearly Explained**. 2017. Vídeo. Disponível em: [https://www.youtube.com/watch?v=HVXime0nQeI](https://www.youtube.com/watch?v=HVXime0nQeI). 

#### **Documentações:**

Este projeto utilizou as seguintes bibliotecas:

**Bibliotecas de Terceiros:**

* **`pandas`:** [https://pandas.pydata.org/docs/](https://pandas.pydata.org/docs/) - Manipulação e análise de dados.
* **`matplotlib.pyplot`:** [https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.html](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.html) - Criação de gráficos estáticos, interativos e animados.
* **`sklearn.preprocessing.StandardScaler`:** [https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) - Padronização de features removendo a média e escalonando para variância unitária.
* **`sklearn.neighbors.KNeighborsRegressor`:** [https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html) - Regressão KNN para prever o valor de novos pontos de dados com base nos vizinhos mais próximos.
* **`sklearn.metrics.mean_squared_error`:** [https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) - Calcula o erro quadrático médio entre os valores reais e previstos.
* **`sklearn.metrics.root_mean_squared_error`:** [https://scikit-learn.org/stable/modules/generated/sklearn.metrics.root_mean_squared_error.html](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.root_mean_squared_error.html) - Calcula a raiz quadrada do erro quadrático médio.

## 👣 **Próximos passos**

Este é o segundo de uma série de 4 notebooks que compõem o trabalho final da disciplina de Aprendizado de Máquina. No próximo caderno, será realizada a etapa de desenvolvimento e avaliação do modelo de aprendizado de máquina da Árvore de Decisão para prever o *band gap* de materiais, e a otimização desse modelo para obter a melhor performance. Para acompanhar o desenvolvimento do projeto, acesse os próximos [notebooks](https://github.com/CaioRuas24010/SepulcroDeDelfos/tree/main/A%20Batalha%20Contra%20Dragao):

1. **[`introducao.ipynb`](https://github.com/CaioRuas24010/SepulcroDeDelfos/blob/main/A%20Batalha%20Contra%20Dragao/introducao.ipynb) - Baixando e organizando os dados**
2. **[`modelo_dos_k-nn_vizinhos.ipynb`](https://github.com/CaioRuas24010/SepulcroDeDelfos/blob/main/A%20Batalha%20Contra%20Dragao/modelo_dos_k-nn_vizinhos.ipynb) - Estudando o modelo k-NN**
3. **[`arvore_de_decisao.ipynb`](https://github.com/CaioRuas24010/SepulcroDeDelfos/blob/main/A%20Batalha%20Contra%20Dragao/arvore_de_decisao.ipynb) - Estudando o modelo Árvore de Decisão**
4. **[`conclusao.ipynb`](https://github.com/CaioRuas24010/SepulcroDeDelfos/blob/main/A%20Batalha%20Contra%20Dragao/conclusao.ipynb) - Resultados e discussões finais** 

---

# **<span style="font-family: 'Palatino Linotype', serif;">Um curto passo de um longo caminho...</span>**

*<span style="font-family: 'Angilla Tattoo'"> <br> Assim, os vizinhos batem na madeira, mas não sabemos mais se é realmente só a da porta. <br> <br> A jornada somente se inicia e outro local até então desconhecido pode ser avistado, não se sabe a procedência e se é somente uma simples e bela árvore. <br> <br> De certa forma, parece que... não! <br> <br> Nossos algoritmos são oráculos, nossos dados são ossos ancestrais. <br> Sepulcro de Delfos </span>*